In [1]:
import tiktoken

In [2]:
with open("harrypotter.txt", "r", encoding="utf-8") as book:
    raw_text = book.read()

tokenizer = tiktoken.get_encoding("gpt2")
encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))

113793


In [3]:
encoded_sample = encoded_text[100:]

# Simple example to create input-target pairs

In [4]:
context_size = 4 # The context size determines how many tokens to include in the input

x = encoded_sample[:context_size]
y = encoded_sample[1:context_size+1]

print(f"x: {x}")
print(f"y: {y}")

x: [20935, 654, 11, 543]
y: [654, 11, 543, 925]


In [6]:
for i in range(context_size):
    x = encoded_sample[:i+1]
    y = encoded_sample[i+1]

    print(f"{x} ---> {y}")

[20935] ---> 654
[20935, 654] ---> 11
[20935, 654, 11] ---> 543
[20935, 654, 11, 543] ---> 925


# Implementing efficient data loader fetching by using PyTorch's DataLoader and Dataset classes

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, context_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenization
        encoded_text = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Using the sliding window appraoch to chunk the book into overlapping sequences of context_length
        for i in range(0, len(encoded_text)-context_length, stride):
            input_chunk = encoded_text[i:i+context_length]
            target_chunk = encoded_text[i+1:i+context_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [16]:
def create_dataloader_v1(txt, batch_size=4, context_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, context_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

# Testing the dataloader with batch_size of 1 for an LLM  with a context_length of 4

In [25]:
with open("harrypotter.txt", "r", encoding="utf-8") as book:
    raw_text = book.read()

In [29]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, context_length=4, stride=1, shuffle=False)

data_iterator = iter(dataloader)
first_batch = next(data_iterator)
first_batch

[tensor([[18308, 14179,   290,   262]]),
 tensor([[14179,   290,   262, 30467]])]

In [30]:
second_batch = next(data_iterator)
second_batch

[tensor([[14179,   290,   262, 30467]]),
 tensor([[  290,   262, 30467,   338]])]

# Testing the dataloader with batch_size of 8 for an LLM  with a context_length of 4

In [31]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, context_length=4, stride=1, shuffle=False)

data_iterator = iter(dataloader)
first_batch = next(data_iterator)
first_batch

[tensor([[18308, 14179,   290,   262],
         [14179,   290,   262, 30467],
         [  290,   262, 30467,   338],
         [  262, 30467,   338,  8026],
         [30467,   338,  8026,   220],
         [  338,  8026,   220,   198],
         [ 8026,   220,   198,   198],
         [  220,   198,   198, 41481]]),
 tensor([[14179,   290,   262, 30467],
         [  290,   262, 30467,   338],
         [  262, 30467,   338,  8026],
         [30467,   338,  8026,   220],
         [  338,  8026,   220,   198],
         [ 8026,   220,   198,   198],
         [  220,   198,   198, 41481],
         [  198,   198, 41481, 16329]])]

***Note:** The input size (context_length) of 4 is relatively very small and only chosen for demonstration purpose in the code. It is common the train LLMs with input size (context_length) of minimum 256* 

<div class="alert-warning">
The small batch size require less memory during training but leads to noisy weight updates.<br/>
Just like in regular deep learning, the batch size is a trade-off and a hyperparameter to experiment with when training LLM.
</div>

# Testing the dataloader with batch_size of 8 for an LLM  with a context_length of 256 (common in training LLMs) and stride of 128

In [32]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, shuffle=False)

data_iterator = iter(dataloader)
first_batch = next(data_iterator)
first_batch

[tensor([[18308, 14179,   290,  ...,   484,  8020,   470],
         [   13,  9074,    13,  ...,  9074,    13,   360],
         [ 1138,   329,  1811,  ...,   319,   262, 19353],
         ...,
         [16809,  9974,    11,  ..., 48508,  6568,   306],
         [  286, 26721,   339,  ...,   736,   284,   262],
         [ 1978,    13,  1770,  ...,   618,   339,  1807]]),
 tensor([[14179,   290,   262,  ...,  8020,   470,  1138],
         [ 9074,    13,   360,  ...,    13,   360,  1834],
         [  329,  1811,   812,  ...,   262, 19353,    11],
         ...,
         [ 9974,    11,   475,  ...,  6568,   306,  1978],
         [26721,   339,   373,  ...,   284,   262,  4324],
         [   13,  1770,    13,  ...,   339,  1807,   339]])]